## import necessary libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer,StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.ensemble import VotingClassifier
%matplotlib inline

## Reading in the dataset

In [2]:
# read in the dataset and preview it
full_dataset=pd.read_csv("train.csv")
full_dataset.head()

,EmployeeNo,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not
0,YAK/S/00001,Commercial Sales and Marketing,"MSc, MBA and PhD",Female,Direct Internal process,2,1986,12.5,2011,1,0,41,ANAMBRA,No,Married,No,No,0,0
1,YAK/S/00002,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,2,1991,12.5,2015,0,0,52,ANAMBRA,Yes,Married,No,No,0,0
2,YAK/S/00003,Commercial Sales and Marketing,First Degree or HND,Male,Direct Internal process,2,1987,7.5,2012,0,0,42,KATSINA,Yes,Married,No,No,0,0
3,YAK/S/00004,Commercial Sales and Marketing,First Degree or HND,Male,Agency and others,3,1982,2.5,2009,0,0,42,NIGER,Yes,Single,No,No,1,0
4,YAK/S/00006,Information and Strategy,First Degree or HND,Male,Direct Internal process,3,1990,7.5,2012,0,0,77,AKWA IBOM,Yes,Married,No,No,1,0


## Slight Exploratory Data Analysis of the data to know what we are working with Plus comments

In [3]:
# get a general feel of the data
full_dataset.info()
# note that the 'qualification' has some missing data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38312 entries, 0 to 38311
Data columns (total 19 columns):
EmployeeNo                             38312 non-null object
Division                               38312 non-null object
Qualification                          36633 non-null object
Gender                                 38312 non-null object
Channel_of_Recruitment                 38312 non-null object
Trainings_Attended                     38312 non-null int64
Year_of_birth                          38312 non-null int64
Last_performance_score                 38312 non-null float64
Year_of_recruitment                    38312 non-null int64
Targets_met                            38312 non-null int64
Previous_Award                         38312 non-null int64
Training_score_average                 38312 non-null int64
State_Of_Origin                        38312 non-null object
Foreign_schooled                       38312 non-null object
Marital_Status                         383

In [4]:
# # visualise some featues that are highly correlated with each other, on the numerical variables
# sns.heatmap(full_dataset.corr())

In [5]:
# # visualise the buckets to see the distribution of the numerical data
# full_dataset.hist(bins=50,figsize=(30,25))
# # side-note convert the ages to respective buckets, there is an outlier that is older than 1960

In [6]:
# get a top-bottom statistical insight into the entire dataset
full_dataset.describe().T

,count,mean,std,min,25%,50%,75%,max
Trainings_Attended,38312.0,2.253680,0.609443,2.0,2.0,2.0,2.0,11.0
Year_of_birth,38312.0,1986.209334,7.646047,1950.0,1982.0,1988.0,1992.0,2001.0
Last_performance_score,38312.0,7.698959,3.744135,0.0,5.0,7.5,10.0,12.5
Year_of_recruitment,38312.0,2013.139695,4.261451,1982.0,2012.0,2014.0,2016.0,2018.0
Targets_met,38312.0,0.352996,0.477908,0.0,0.0,0.0,1.0,1.0
Previous_Award,38312.0,0.023152,0.150388,0.0,0.0,0.0,0.0,1.0
Training_score_average,38312.0,55.366465,13.362741,31.0,43.0,52.0,68.0,91.0
Promoted_or_Not,38312.0,0.084595,0.278282,0.0,0.0,0.0,0.0,1.0


In [7]:
# # check out the people who were born before 1960, since they are so few(12), we then turn them into category "people < 1960"
# # the age is also normally distributed/slightly tail heavy,so there is no need for much transformation.
# full_dataset[full_dataset['Year_of_birth']<1960].shape[0]
# # min 1950   max 2001

In [8]:
#full_dataset['No_of_previous_employers'].value_counts()

In [9]:
full_dataset['No_of_previous_employers'] = full_dataset['No_of_previous_employers'].replace('More_than_5', 6)
full_dataset['Marital_Status'] = full_dataset['Marital_Status'].replace('Not_Sure', 'Single')
full_dataset['Qualification'] = full_dataset['Qualification'].replace(np.NaN, 'Non-University Education')

## Indepth Exploratory data analysis and Sidenotes

In [10]:
full_dataset.head(2)

,EmployeeNo,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not
0,YAK/S/00001,Commercial Sales and Marketing,"MSc, MBA and PhD",Female,Direct Internal process,2,1986,12.5,2011,1,0,41,ANAMBRA,No,Married,No,No,0,0
1,YAK/S/00002,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,2,1991,12.5,2015,0,0,52,ANAMBRA,Yes,Married,No,No,0,0


In [11]:
# full_dataset['Marital_Status'].value_counts()
# # turn any not-sure into Single to prevent our algorithm from learning garbage

In [12]:
# full_dataset['State_Of_Origin'].value_counts()
# # i should probably map state of each state to the three major tribes to know if they are yoruba, igbo or hausa

In [13]:
full_dataset['Trainings_Attended'].value_counts()
# since greater than 5 training is pretty few, i should consider making it anything>5 = 5

2     30981
3      5631
4      1244
5       316
6        93
7        28
8         6
10        5
11        4
9         4
Name: Trainings_Attended, dtype: int64

In [14]:
full_dataset['Channel_of_Recruitment'].value_counts()
# nothing here too. we just categorically encode them

Agency and others                  21310
Direct Internal process            16194
Referral and Special candidates      808
Name: Channel_of_Recruitment, dtype: int64

In [15]:
full_dataset['Gender'].value_counts()
# genders are fair, i guess. nothing much to see here, we just simply categorically encode it

Male      26880
Female    11432
Name: Gender, dtype: int64

In [16]:
full_dataset['Qualification'].value_counts()
# this seems pretty distributed too, but we categorically encode them wrt their level 3,2,1
# fill in missing values with Non-University Education

First Degree or HND         25578
MSc, MBA and PhD            10469
Non-University Education     2265
Name: Qualification, dtype: int64

In [17]:
full_dataset['Division'].value_counts()
# pretty well distributed, no further engineering needed, simply categorically encoding them

Commercial Sales and Marketing                 11695
Customer Support and Field Operations           7973
Sourcing and Purchasing                         5052
Information Technology and Solution Support     4952
Information and Strategy                        3721
Business Finance Operations                     1786
People/HR Management                            1704
Regulatory and Legal services                    733
Research and Innovation                          696
Name: Division, dtype: int64

In [18]:
# note that the dataset it gr
full_dataset['Promoted_or_Not'].value_counts()

0    35071
1     3241
Name: Promoted_or_Not, dtype: int64

## Data Spliting

#### Split into Train,Testing and validation set (60%,25%,15%)

In [19]:
X=full_dataset.drop(['Promoted_or_Not'],1)
y=full_dataset['Promoted_or_Not']

In [20]:
# split into train and test/validation
X_train, X_test_valid, y_train, y_test_valid = train_test_split(X, y, test_size=0.4, random_state=42)

# split into test and validation
X_test,X_validation,y_test,y_validation=train_test_split(X_test_valid,y_test_valid,test_size=0.375)

In [21]:
# size of my batches of data
print("train size of rows:{} and columns:{}".format(*X_train.shape))
print("train size of rows:{} and columns:{}".format(*X_test.shape))
print("train size of rows:{} and columns:{}".format(*X_validation.shape))

train size of rows:22987 and columns:18
train size of rows:9578 and columns:18
train size of rows:5747 and columns:18


## Custom mappings we can use to transform our data

In [22]:
# manually curated mapping for categorical variables, these values were derived from my intuition and domain knowledge
qualification_to_number={'First Degree or HND':2, 'MSc, MBA and PhD':3, 'Non-University Education':1}
genderMap={"Male":1.1,"Female":3.5}
yesnoMap={"Yes":1,"No":0}
noyesMap={"Yes":0,"No":1}
states_to_tribe={
    'ABIA': "IGBO",
    'ADAMAWA':"HAUSA",
    'AKWA IBOM':"IGBO",
    'ANAMBRA':"IGBO",
    'BAUCHI':"IGBO",
    'BAYELSA':"IGBO",
    'BENUE':"HAUSA",
    'BORNO':"HAUSA",
    'CROSS RIVER':"IGBO",
    'DELTA':"IGBO",
    'EBONYI':"IGBO",
    'EDO':"IGBO",
    'EKITI':"YORUBA",
    'ENUGU':"IGBO",
    'FCT':"HAUSA",
    'GOMBE':"HAUSA",
    'IMO':"IGBO",
    'JIGAWA':"HAUSA",
    'KADUNA':"HAUSA",
    'KANO':'HAUSA',
    'KATSINA':"HAUSA",
    'KEBBI':"HAUSA",
    'KOGI':"HAUSA",
    'KWARA':"HAUSA",
    "LAGOS":"YORUBA",
    'NASSARAWA':"HAUSA",
    'NIGER':"HAUSA",
     'OGUN':"YORUBA",
     'ONDO':"YORUBA",
     'OSUN':"YORUBA",
     'OYO':"YORUBA",
     'PLATEAU':"HAUSA",
     'RIVERS':"IGBO",
     'SOKOTO':"HAUSA",
     'TARABA':"HAUSA",
     'YOBE':"HAUSA",
     'ZAMFARA':"HAUSA"
    }
    
dept_to_number={
    "Commercial Sales and Marketing":50.0,
    "Customer Support and Field Operations":31.4,
    "Business Finance Operations":30.7,
    "People/HR Management":30.4,
    "Sourcing and Purchasing":19.5,
    "Regulatory and Legal services":18.0,
    "Information Technology and Solution Support":7.3,
    "Information and Strategy":5.1,
    "Research and Innovation":4.3
}
maritalMap={"Married":1,"Single":0.5}

def map_employers(x):
    try: 
        return int(x)
    except:
        return 5

In [23]:
# create a mappinng function which returns a dictionary mapping of the average values of that category which are promoted
# this was developed out of sheer desperation to try and boost my score :/

def mapCategoricalToAverage(name):
    mapping=dict(full_dataset.groupby(name)['Promoted_or_Not'].mean())
    return lambda x: mapping[x]

## Specify my data transformations

In [24]:
from sklearn.base import BaseEstimator,TransformerMixin
full_dataset.head(3)

,EmployeeNo,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not
0,YAK/S/00001,Commercial Sales and Marketing,"MSc, MBA and PhD",Female,Direct Internal process,2,1986,12.5,2011,1,0,41,ANAMBRA,No,Married,No,No,0,0
1,YAK/S/00002,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,2,1991,12.5,2015,0,0,52,ANAMBRA,Yes,Married,No,No,0,0
2,YAK/S/00003,Commercial Sales and Marketing,First Degree or HND,Male,Direct Internal process,2,1987,7.5,2012,0,0,42,KATSINA,Yes,Married,No,No,0,0


In [26]:
# specify the numeric columns we dont want to transform
numeric_columns=['Trainings_Attended', 'Year_of_birth', 'Last_performance_score', 'Year_of_recruitment', 'Targets_met', 'Previous_Award', 'Training_score_average']
class myTransformer():
    def __init__(self):
        self.division_binarizer=LabelBinarizer()
        self.channel_binariser=LabelBinarizer()
        self.tribe_binariser=LabelBinarizer()
        self.dataScaler=StandardScaler()
    
    # Fit all the binarisers on the training data
    def fit(self,input_data):
        self.division_binarizer=self.division_binarizer.fit(input_data['Division'])
        self.channel_binariser=self.channel_binariser.fit(input_data['Channel_of_Recruitment'])
        self.tribe_binariser=self.tribe_binariser.fit(input_data['State_Of_Origin'])
        
    # Transform the input data using the fitted binarisers
    def transform(self,full_dataset,train=False):
        #     making a copy of the input because we dont want to change the input in the main function
        input_data=full_dataset.copy()
        # label binarise the dvision
        division_binarised=self.division_binarizer.transform(input_data['Division'])

        # categorise the qualifications
        input_data['Qualification']=input_data['Qualification'].fillna("Non-University Education")
        qualification_num=input_data['Qualification'].map(mapCategoricalToAverage("Qualification")).values

        # categorise the gender
        gender_num=input_data["Gender"].map(mapCategoricalToAverage("Gender")).values

        # binarise the channel
        channel_binarised=self.channel_binariser.transform(input_data['Channel_of_Recruitment'])

        # map state of origin to tribe and binarise it
        state_binarised=self.tribe_binariser.transform(input_data['State_Of_Origin'])

        # map foreign schooled
        foreign_schooled_num=input_data['Foreign_schooled'].map(mapCategoricalToAverage("Foreign_schooled")).values

        # map marital status
#         marital_status_num=input_data['Marital_Status'].map(lambda x: "Single" if x=="Not_Sure" else x).map(maritalMap).values

        # map past disciplinary actions
        past_discipline_num=input_data['Past_Disciplinary_Action'].map(mapCategoricalToAverage("Past_Disciplinary_Action")).values

        # map interdep movement
        interdep_movement_num=input_data['Previous_IntraDepartmental_Movement'].map(mapCategoricalToAverage("Previous_IntraDepartmental_Movement")).values

        # map employer
        previous_employer_count=input_data['No_of_previous_employers'].map(mapCategoricalToAverage("No_of_previous_employers")).values

        numeric_data=input_data[numeric_columns].values
        
        # Create new variables
        qualification_times_scoreavg=(qualification_num*input_data['Training_score_average']).values
        department_times_scoreavg=(input_data['Division'].map(dept_to_number) * input_data['Training_score_average']).values
        department_in_number=input_data['Division'].map(dept_to_number)

        
        # this concatenates all the data
        fully_transformed=np.c_[qualification_times_scoreavg,department_times_scoreavg,department_in_number,division_binarised,qualification_num,gender_num,channel_binarised,state_binarised,foreign_schooled_num,past_discipline_num,interdep_movement_num,previous_employer_count,numeric_data]
#         fully_transformed=np.c_[division_binarised,qualification_num,gender_num,channel_binarised,state_binarised,foreign_schooled_num,marital_status_num,past_discipline_num,interdep_movement_num,previous_employer_count,numeric_data]
        return fully_transformed

In [27]:
full_dataset.head()

,EmployeeNo,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not
0,YAK/S/00001,Commercial Sales and Marketing,"MSc, MBA and PhD",Female,Direct Internal process,2,1986,12.5,2011,1,0,41,ANAMBRA,No,Married,No,No,0,0
1,YAK/S/00002,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,2,1991,12.5,2015,0,0,52,ANAMBRA,Yes,Married,No,No,0,0
2,YAK/S/00003,Commercial Sales and Marketing,First Degree or HND,Male,Direct Internal process,2,1987,7.5,2012,0,0,42,KATSINA,Yes,Married,No,No,0,0
3,YAK/S/00004,Commercial Sales and Marketing,First Degree or HND,Male,Agency and others,3,1982,2.5,2009,0,0,42,NIGER,Yes,Single,No,No,1,0
4,YAK/S/00006,Information and Strategy,First Degree or HND,Male,Direct Internal process,3,1990,7.5,2012,0,0,77,AKWA IBOM,Yes,Married,No,No,1,0


## Spliting the training and testing and validation data

In [28]:
input_dataset=full_dataset.drop("Promoted_or_Not",1)
output_dataset=full_dataset["Promoted_or_Not"]

In [29]:
x_train,x_test_valid,y_train,y_test_valid=train_test_split(input_dataset,output_dataset,test_size=0.35,random_state=42)
x_test,x_valid,y_test,y_valid=train_test_split(x_test_valid,y_test_valid,test_size=0.3,random_state=42)

## Transforming the train_set and the test_set and validation_set

In [30]:
transformer=myTransformer()
transformer.fit(x_train)

In [31]:
transformed_x_train=transformer.transform(x_train,train=True)
transformed_x_test=transformer.transform(x_test)
transformed_x_valid=transformer.transform(x_valid)

In [33]:
pd.value_counts(y_train)

0    22785
1     2117
Name: Promoted_or_Not, dtype: int64

#### the ratio of 10 to 0 is so imbalanced, we need to balance them by augumenting the data

In [33]:
transformed_x_train.shape

(24902, 65)

## Augumenting the imbalanced dataset

In [34]:
# use adasyn to balance the dataset
def makeOverSamplesADASYN(X,y):
 #input DataFrame
 #X →Independent Variable in DataFrame\
 #y →dependent Variable in Pandas DataFrame format
     from imblearn.over_sampling import ADASYN 
     sm = ADASYN()
     X,y = sm.fit_sample(X, y)
     return(X,y)
    
balanced_x_train,balanced_y_train=makeOverSamplesADASYN(transformed_x_train,y_train)

/home/godwin/.local/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/home/godwin/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/home/godwin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In futu

In [35]:
pd.value_counts(balanced_y_train)

1    23160
0    22785
dtype: int64

#### Finally, our training set is now balanced as the values of 1 and 0 are almost equal

## Finally training the model

#### using xg-boost as my base model

In [36]:
from xgboost import XGBClassifier
from sklearn import metrics

In [38]:
clf = XGBClassifier(base_score=0.7,booster="dart",n_estimators=3000,
                    max_depth=8,learning_rate=0.01,objective='binary:logistic',subsample=0.9,reg_lambda=0.03)
eval_set  = [(transformed_x_train,y_train), (transformed_x_test,y_test)]
#.900057
clf.fit(transformed_x_train, y_train, eval_set=eval_set,eval_metric="auc", early_stopping_rounds=200)

[0]	validation_0-auc:0.895667	validation_1-auc:0.891077
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 200 rounds.
[1]	validation_0-auc:0.900314	validation_1-auc:0.893303
[2]	validation_0-auc:0.902987	validation_1-auc:0.895238
[3]	validation_0-auc:0.9032	validation_1-auc:0.895972
[4]	validation_0-auc:0.90465	validation_1-auc:0.89776
[5]	validation_0-auc:0.905684	validation_1-auc:0.897185
[6]	validation_0-auc:0.9063	validation_1-auc:0.897223
[7]	validation_0-auc:0.906119	validation_1-auc:0.896797
[8]	validation_0-auc:0.906769	validation_1-auc:0.899873
[9]	validation_0-auc:0.908206	validation_1-auc:0.901107
[10]	validation_0-auc:0.908358	validation_1-auc:0.900773
[11]	validation_0-auc:0.908408	validation_1-auc:0.900497
[12]	validation_0-auc:0.908132	validation_1-auc:0.900344
[13]	validation_0-auc:0.908357	validation_1-auc:0.900099
[14]	validation_0-auc:0.908281	validation_1-auc:0.900102
[15

[142]	validation_0-auc:0.923785	validation_1-auc:0.908218
[143]	validation_0-auc:0.923926	validation_1-auc:0.908238
[144]	validation_0-auc:0.924019	validation_1-auc:0.908265
[145]	validation_0-auc:0.924025	validation_1-auc:0.90822
[146]	validation_0-auc:0.924089	validation_1-auc:0.908346
[147]	validation_0-auc:0.924177	validation_1-auc:0.908245
[148]	validation_0-auc:0.924217	validation_1-auc:0.908205
[149]	validation_0-auc:0.924221	validation_1-auc:0.908174
[150]	validation_0-auc:0.924331	validation_1-auc:0.908083
[151]	validation_0-auc:0.924593	validation_1-auc:0.908086
[152]	validation_0-auc:0.924709	validation_1-auc:0.908144
[153]	validation_0-auc:0.924741	validation_1-auc:0.90828
[154]	validation_0-auc:0.924843	validation_1-auc:0.908117
[155]	validation_0-auc:0.92494	validation_1-auc:0.908112
[156]	validation_0-auc:0.925144	validation_1-auc:0.908179
[157]	validation_0-auc:0.925197	validation_1-auc:0.908135
[158]	validation_0-auc:0.925272	validation_1-auc:0.908098
[159]	validation_

[284]	validation_0-auc:0.938979	validation_1-auc:0.90963
[285]	validation_0-auc:0.939142	validation_1-auc:0.90965
[286]	validation_0-auc:0.939295	validation_1-auc:0.909595
[287]	validation_0-auc:0.9394	validation_1-auc:0.909573
[288]	validation_0-auc:0.939489	validation_1-auc:0.909612
[289]	validation_0-auc:0.939658	validation_1-auc:0.9096
[290]	validation_0-auc:0.939746	validation_1-auc:0.909575
[291]	validation_0-auc:0.939888	validation_1-auc:0.909589
[292]	validation_0-auc:0.939911	validation_1-auc:0.909591
[293]	validation_0-auc:0.940171	validation_1-auc:0.909679
[294]	validation_0-auc:0.94041	validation_1-auc:0.909709
[295]	validation_0-auc:0.940495	validation_1-auc:0.909746
[296]	validation_0-auc:0.940515	validation_1-auc:0.909788
[297]	validation_0-auc:0.940628	validation_1-auc:0.909791
[298]	validation_0-auc:0.940691	validation_1-auc:0.909795
[299]	validation_0-auc:0.940737	validation_1-auc:0.909724
[300]	validation_0-auc:0.94094	validation_1-auc:0.909781
[301]	validation_0-auc

[426]	validation_0-auc:0.954107	validation_1-auc:0.909485
[427]	validation_0-auc:0.954228	validation_1-auc:0.909475
[428]	validation_0-auc:0.95443	validation_1-auc:0.90948
[429]	validation_0-auc:0.954483	validation_1-auc:0.909482
[430]	validation_0-auc:0.954546	validation_1-auc:0.909438
[431]	validation_0-auc:0.954672	validation_1-auc:0.909462
[432]	validation_0-auc:0.954713	validation_1-auc:0.909425
[433]	validation_0-auc:0.954777	validation_1-auc:0.909428
[434]	validation_0-auc:0.954859	validation_1-auc:0.909392
[435]	validation_0-auc:0.954935	validation_1-auc:0.909285
[436]	validation_0-auc:0.954984	validation_1-auc:0.909305
[437]	validation_0-auc:0.9551	validation_1-auc:0.909303
[438]	validation_0-auc:0.955153	validation_1-auc:0.909292
[439]	validation_0-auc:0.955209	validation_1-auc:0.909311
[440]	validation_0-auc:0.955283	validation_1-auc:0.909257
[441]	validation_0-auc:0.955413	validation_1-auc:0.909245
[442]	validation_0-auc:0.955509	validation_1-auc:0.909265
[443]	validation_0

XGBClassifier(base_score=0.7, booster='dart', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.01, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=None, n_estimators=3000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=0.03, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.9, verbosity=1)

#### testing the xgboost model

In [39]:
y_train_pred = clf.predict(transformed_x_train)
# how did our model perform on the train set?
count_misclassified = (y_train != y_train_pred).sum()
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = metrics.accuracy_score(y_train_pred, y_train)
print('Accuracy: {:.2f}'.format(accuracy))

Misclassified samples: 1324
Accuracy: 0.95


In [40]:
y_valid_pred = clf.predict(transformed_x_valid)
# how did our model perform on the test set?
count_misclassified = (y_valid != y_valid_pred).sum()
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = metrics.accuracy_score(y_valid_pred, y_valid)
print('Accuracy: {:.2f}'.format(accuracy))

Misclassified samples: 251
Accuracy: 0.94


## checking out lgmboost

In [41]:
import lightgbm 
lightgbm.LGBMClassifier()

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [42]:
gbm = lightgbm.LGBMClassifier(boosting_type="dart",n_estimators=10000,learning_rate=0.01,num_leaves = 15,max_depth=6,subsample=0.9
                              ,colsample_bytree=0.3,reg_lambda=0.9,early_stopping_rounds=50)

In [43]:
gbm.fit(balanced_x_train,balanced_y_train,eval_metric='auc',eval_set=[(transformed_x_test, y_test)],early_stopping_rounds=10000)

/home/godwin/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's binary_logloss: 0.696125	valid_0's auc: 0.633735
[2]	valid_0's binary_logloss: 0.691598	valid_0's auc: 0.635567
[3]	valid_0's binary_logloss: 0.687945	valid_0's auc: 0.688624
[4]	valid_0's binary_logloss: 0.684678	valid_0's auc: 0.719198
[5]	valid_0's binary_logloss: 0.681071	valid_0's auc: 0.710169
[6]	valid_0's binary_logloss: 0.677343	valid_0's auc: 0.708628
[7]	valid_0's binary_logloss: 0.672616	valid_0's auc: 0.78132
[8]	valid_0's binary_logloss: 0.673626	valid_0's auc: 0.797093
[9]	valid_0's binary_logloss: 0.669661	valid_0's auc: 0.791778
[10]	valid_0's binary_logloss: 0.667194	valid_0's auc: 0.795163
[11]	valid_0's binary_logloss: 0.663614	valid_0's auc: 0.785639
[12]	valid_0's binary_logloss: 0.665109	valid_0's auc: 0.793385
[13]	valid_0's binary_logloss: 0.661945	valid_0's auc: 0.789151
[14]	valid_0's binary_logloss: 0.658614	valid_0's auc: 0.784519
[15]	valid_0's binary_logloss: 0.654666	valid_0's auc: 0.788643
[16]	valid_0's binary_logloss: 0.651266	valid_0's 

/home/godwin/anaconda3/lib/python3.6/site-packages/lightgbm/callback.py:189: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')


[20]	valid_0's binary_logloss: 0.636912	valid_0's auc: 0.816908
[21]	valid_0's binary_logloss: 0.637507	valid_0's auc: 0.815314
[22]	valid_0's binary_logloss: 0.63381	valid_0's auc: 0.814088
[23]	valid_0's binary_logloss: 0.630481	valid_0's auc: 0.814769
[24]	valid_0's binary_logloss: 0.62779	valid_0's auc: 0.819083
[25]	valid_0's binary_logloss: 0.62463	valid_0's auc: 0.818997
[26]	valid_0's binary_logloss: 0.619939	valid_0's auc: 0.842441
[27]	valid_0's binary_logloss: 0.6167	valid_0's auc: 0.840257
[28]	valid_0's binary_logloss: 0.617877	valid_0's auc: 0.840167
[29]	valid_0's binary_logloss: 0.614923	valid_0's auc: 0.839487
[30]	valid_0's binary_logloss: 0.610675	valid_0's auc: 0.843571
[31]	valid_0's binary_logloss: 0.610741	valid_0's auc: 0.841447
[32]	valid_0's binary_logloss: 0.606838	valid_0's auc: 0.841236
[33]	valid_0's binary_logloss: 0.604187	valid_0's auc: 0.841317
[34]	valid_0's binary_logloss: 0.600349	valid_0's auc: 0.839597
[35]	valid_0's binary_logloss: 0.60189	valid_

[148]	valid_0's binary_logloss: 0.539284	valid_0's auc: 0.850284
[149]	valid_0's binary_logloss: 0.536172	valid_0's auc: 0.851068
[150]	valid_0's binary_logloss: 0.534011	valid_0's auc: 0.851007
[151]	valid_0's binary_logloss: 0.535189	valid_0's auc: 0.850837
[152]	valid_0's binary_logloss: 0.536305	valid_0's auc: 0.850836
[153]	valid_0's binary_logloss: 0.534666	valid_0's auc: 0.850289
[154]	valid_0's binary_logloss: 0.535598	valid_0's auc: 0.850371
[155]	valid_0's binary_logloss: 0.532417	valid_0's auc: 0.850363
[156]	valid_0's binary_logloss: 0.533726	valid_0's auc: 0.850218
[157]	valid_0's binary_logloss: 0.534352	valid_0's auc: 0.850397
[158]	valid_0's binary_logloss: 0.535652	valid_0's auc: 0.850247
[159]	valid_0's binary_logloss: 0.537024	valid_0's auc: 0.850512
[160]	valid_0's binary_logloss: 0.53838	valid_0's auc: 0.850284
[161]	valid_0's binary_logloss: 0.535393	valid_0's auc: 0.848438
[162]	valid_0's binary_logloss: 0.536154	valid_0's auc: 0.848173
[163]	valid_0's binary_log

...

[9616]	valid_0's binary_logloss: 0.162831	valid_0's auc: 0.904013
[9617]	valid_0's binary_logloss: 0.162833	valid_0's auc: 0.904017
[9618]	valid_0's binary_logloss: 0.162832	valid_0's auc: 0.904012
[9619]	valid_0's binary_logloss: 0.162836	valid_0's auc: 0.904003
[9620]	valid_0's binary_logloss: 0.162838	valid_0's auc: 0.904003
[9621]	valid_0's binary_logloss: 0.162837	valid_0's auc: 0.903994
[9622]	valid_0's binary_logloss: 0.162837	valid_0's auc: 0.903992
[9623]	valid_0's binary_logloss: 0.16284	valid_0's auc: 0.903992
[9624]	valid_0's binary_logloss: 0.162842	valid_0's auc: 0.903991
[9625]	valid_0's binary_logloss: 0.162843	valid_0's auc: 0.90399
[9626]	valid_0's binary_logloss: 0.162845	valid_0's auc: 0.903988
[9627]	valid_0's binary_logloss: 0.162848	valid_0's auc: 0.903979
[9628]	valid_0's binary_logloss: 0.16285	valid_0's auc: 0.903964
[9629]	valid_0's binary_logloss: 0.162854	valid_0's auc: 0.903963
[9630]	valid_0's binary_logloss: 0.162855	valid_0's auc: 0.903962
[9631]	valid_

[9743]	valid_0's binary_logloss: 0.162798	valid_0's auc: 0.903949
[9744]	valid_0's binary_logloss: 0.1628	valid_0's auc: 0.903949
[9745]	valid_0's binary_logloss: 0.162796	valid_0's auc: 0.903946
[9746]	valid_0's binary_logloss: 0.162798	valid_0's auc: 0.903947
[9747]	valid_0's binary_logloss: 0.1628	valid_0's auc: 0.903946
[9748]	valid_0's binary_logloss: 0.162801	valid_0's auc: 0.903947
[9749]	valid_0's binary_logloss: 0.1628	valid_0's auc: 0.90395
[9750]	valid_0's binary_logloss: 0.162801	valid_0's auc: 0.90395
[9751]	valid_0's binary_logloss: 0.162803	valid_0's auc: 0.903949
[9752]	valid_0's binary_logloss: 0.162805	valid_0's auc: 0.903949
[9753]	valid_0's binary_logloss: 0.162806	valid_0's auc: 0.903943
[9754]	valid_0's binary_logloss: 0.162803	valid_0's auc: 0.90395
[9755]	valid_0's binary_logloss: 0.162805	valid_0's auc: 0.903942
[9756]	valid_0's binary_logloss: 0.162801	valid_0's auc: 0.903943
[9757]	valid_0's binary_logloss: 0.162795	valid_0's auc: 0.903945
[9758]	valid_0's bi

[9870]	valid_0's binary_logloss: 0.162748	valid_0's auc: 0.904042
[9871]	valid_0's binary_logloss: 0.162749	valid_0's auc: 0.904043
[9872]	valid_0's binary_logloss: 0.162751	valid_0's auc: 0.904043
[9873]	valid_0's binary_logloss: 0.162748	valid_0's auc: 0.904048
[9874]	valid_0's binary_logloss: 0.162744	valid_0's auc: 0.904042
[9875]	valid_0's binary_logloss: 0.162737	valid_0's auc: 0.904052
[9876]	valid_0's binary_logloss: 0.162731	valid_0's auc: 0.904063
[9877]	valid_0's binary_logloss: 0.162734	valid_0's auc: 0.904052
[9878]	valid_0's binary_logloss: 0.162738	valid_0's auc: 0.904041
[9879]	valid_0's binary_logloss: 0.162731	valid_0's auc: 0.904045
[9880]	valid_0's binary_logloss: 0.162732	valid_0's auc: 0.904047
[9881]	valid_0's binary_logloss: 0.162734	valid_0's auc: 0.904046
[9882]	valid_0's binary_logloss: 0.162737	valid_0's auc: 0.904047
[9883]	valid_0's binary_logloss: 0.16274	valid_0's auc: 0.904045
[9884]	valid_0's binary_logloss: 0.162729	valid_0's auc: 0.904043
[9885]	vali

[9995]	valid_0's binary_logloss: 0.162668	valid_0's auc: 0.904133
[9996]	valid_0's binary_logloss: 0.162669	valid_0's auc: 0.904131
[9997]	valid_0's binary_logloss: 0.162668	valid_0's auc: 0.904129
[9998]	valid_0's binary_logloss: 0.162666	valid_0's auc: 0.904127
[9999]	valid_0's binary_logloss: 0.162668	valid_0's auc: 0.904128
[10000]	valid_0's binary_logloss: 0.162669	valid_0's auc: 0.904126


LGBMClassifier(boosting_type='dart', class_weight=None, colsample_bytree=0.3,
               early_stopping_rounds=50, importance_type='split',
               learning_rate=0.01, max_depth=6, min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=10000,
               n_jobs=-1, num_leaves=15, objective=None, random_state=None,
               reg_alpha=0.0, reg_lambda=0.9, silent=True, subsample=0.9,
               subsample_for_bin=200000, subsample_freq=0)

#### testing the model on the train set

In [46]:
from sklearn import metrics
y_train_pred = gbm.predict(transformed_x_train)
# how did our model perform on the train set?
count_misclassified = (y_train != y_train_pred).sum()
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = metrics.accuracy_score(y_train_pred, y_train)
print('Accuracy: {:.2f}'.format(accuracy))

Misclassified samples: 1348
Accuracy: 0.95


#### testing on the test set

In [47]:
y_valid_pred = gbm.predict(transformed_x_valid)
# how did our model perform on the test set?
count_misclassified = (y_valid != y_valid_pred).sum()
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = metrics.accuracy_score(y_valid_pred, y_valid)
print('Accuracy: {:.2f}'.format(accuracy))

Misclassified samples: 250
Accuracy: 0.94


In [112]:
lbgo=y_valid_pred

0.9965200099428287

## checking out catboost

In [48]:
from catboost import CatBoostClassifier, Pool

ModuleNotFoundError: No module named 'catboost'

In [ ]:
test_pool = Pool(transformed_x_test)

In [43]:
eval_dataset = Pool(transformed_x_test, y_test)

catb_model = CatBoostClassifier(learning_rate=0.01,eval_metric='AUC',depth=6,bagging_temperature=0.15)
#904623 d=6
# catb_model.fit(transformed_x_train,
#           y_train,
#           eval_set=eval_dataset,
#           verbose=True)

#### test cat boost on the training set

In [107]:
y_train_pred = catb_model.predict(transformed_x_train)
# how did our model perform on the train set?
count_misclassified = (y_train != y_train_pred).sum()
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = metrics.accuracy_score(y_train_pred, y_train)
print('Accuracy: {:.2f}'.format(accuracy))

Misclassified samples: 1382
Accuracy: 0.94


#### test cat boost on the validation set

In [115]:
y_valid_pred = catb_model.predict(transformed_x_valid)
# how did our model perform on the test set?
count_misclassified = (y_valid != y_valid_pred).sum()
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = metrics.accuracy_score(y_valid_pred, y_valid)
print('Accuracy: {:.2f}'.format(accuracy))

Misclassified samples: 253
Accuracy: 0.94


In [116]:
catgbo=y_valid_pred

#### On the verge of giving up due to fraustration and mental fatigue, in sheer desperation i have resorted to a randomForest model